In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Tuple, Dict, Optional
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
import json
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, SimpleRNN, Conv1D, Flatten, MaxPooling1D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Note

This uses datasets used in the paper below:

[Analysis and prediciton of atmospheric ozone concentrations using machine learning](https://www.researchgate.net/publication/388531615_Analysis_and_prediction_of_atmospheric_ozone_concentrations_using_machine_learning/fulltext/67b1263e645ef274a481f1b1/Analysis-and-prediction-of-atmospheric-ozone-concentrations-using-machine-learning.pdf)

## Linear Model

In [ ]:
"""
Extended version of the NABEL data loader that includes all parameters
needed for the non-linear ozone prediction model (Model 4).
"""

class ExtendedNABELDataLoader:
    def __init__(self, data_dir: str = "/content/drive/MyDrive/MBD-FRP/data/raw"):
        """
        Initialize the extended NABEL data loader.

        Args:
            data_dir (str): Directory where raw NABEL data files are stored
        """
        self.data_dir = data_dir
        self.station_name = "Lugano-Università"
        self.required_params = ['O3', 'NO2', 'NOX', 'SO2', 'NMVOC', 'TEMP', 'RAD']
        self.all_params = self.required_params + ['CO', 'PM10', 'PM2.5', 'EC', 'CPC', 'PREC']

    def load_parameter_file(self, param_name: str) -> Optional[pd.DataFrame]:
        """
        Load data for a specific parameter from CSV file.

        Args:
            param_name (str): Name of parameter to load

        Returns:
            Optional[pd.DataFrame]: DataFrame with date and parameter value, or None if file not found
        """
        file_path = os.path.join(self.data_dir, f"{param_name}.csv")
        if not os.path.exists(file_path):
            print(f"Warning: File not found for parameter {param_name}")
            return None

        try:
            # Load file, skip header rows and rename columns
            df = pd.read_csv(file_path, sep=';', skiprows=4, encoding='latin1')

            # Extract only first two columns (date and value)
            df = df.iloc[:, :2]

            # Rename columns
            df.columns = ['date', param_name.lower()]

            # Convert date to datetime
            df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')

            # Convert values to numeric, handling any non-numeric values
            df[param_name.lower()] = pd.to_numeric(df[param_name.lower()].astype(str).str.replace(',', '.'),
                                                errors='coerce')

            return df

        except Exception as e:
            print(f"Error loading {param_name} data: {e}")
            return None

    def load_and_prepare_data(self, start_year: int = 2016,
                            end_year: int = 2023) -> pd.DataFrame:
        """
        Load and prepare NABEL data for the specified time period.

        Args:
            start_year (int): Start year for data loading
            end_year (int): End year for data loading

        Returns:
            pd.DataFrame: Prepared dataset with daily averages
        """
        # Load data for each parameter
        data_frames = {}
        for param in self.all_params:
            df = self.load_parameter_file(param)
            if df is not None:
                data_frames[param] = df

        # Check if all required parameters are loaded
        missing_required = [param for param in self.required_params if param not in data_frames]
        if missing_required:
            raise ValueError(f"Missing required parameters: {missing_required}")

        # Start with ozone data
        merged_df = data_frames['O3']

        # Merge with other parameters
        for param, df in data_frames.items():
            if param != 'O3':
                merged_df = pd.merge(merged_df, df, on='date', how='inner')

        # Filter by date range
        mask = (merged_df['date'].dt.year >= start_year) & (merged_df['date'].dt.year <= end_year)
        merged_df = merged_df[mask].copy()

        # Sort by date
        merged_df = merged_df.sort_values('date')

        # Rename columns to standard format expected by models
        column_mapping = {
            'o3': 'ozone',
            'rad': 'RAD',
            'no2': 'NO2',
            'nox': 'NOX',
            'so2': 'SO2',
            'nmvoc': 'NMVOC',
            'temp': 'TEMP',
            'co': 'CO',
            'pm10': 'PM10',
            'pm2.5': 'PM2.5',
            'ec': 'EC',
            'cpc': 'CPC',
            'prec': 'PREC'
        }

        merged_df = merged_df.rename(columns={k: v for k, v in column_mapping.items()
                                            if k in merged_df.columns})

        return merged_df

    def process_raw_data(self, df: pd.DataFrame, handle_missing: str = 'drop') -> pd.DataFrame:
        """
        Process raw NABEL data.

        Args:
            df (pd.DataFrame): Raw data from NABEL
            handle_missing (str): Strategy for handling missing values ('drop' or 'interpolate')

        Returns:
            pd.DataFrame: Processed data
        """
        # Handle missing values
        if handle_missing == 'drop':
            # Simply drop rows with any missing values
            df = df.dropna()
        elif handle_missing == 'interpolate':
            # Interpolate missing values
            df = df.set_index('date')
            df = df.interpolate(method='time')
            df = df.reset_index()
        else:
            raise ValueError(f"Invalid handle_missing strategy: {handle_missing}")

        # Add derived columns that might be useful
        df['Year'] = df['date'].dt.year
        df['Month'] = df['date'].dt.month
        df['DayOfYear'] = df['date'].dt.dayofyear

        # Add seasonality features (based on day of year)
        df['Season_Sin'] = np.sin(2 * np.pi * df['DayOfYear'] / 365)
        df['Season_Cos'] = np.cos(2 * np.pi * df['DayOfYear'] / 365)

        return df

    def split_data(self, df: pd.DataFrame,
                  test_size: float = 0.2,
                  random_state: int = 42,
                  time_based: bool = True) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Split data into training and testing sets.

        Args:
            df (pd.DataFrame): Processed daily data
            test_size (float): Proportion of data to use for testing
            random_state (int): Random seed for reproducibility
            time_based (bool): Whether to split based on time (True) or randomly (False)

        Returns:
            Tuple[pd.DataFrame, pd.DataFrame]: Training and testing datasets
        """
        if time_based:
            # Split based on time (maintain temporal order)
            split_idx = int(len(df) * (1 - test_size))
            train_data = df.iloc[:split_idx].copy()
            test_data = df.iloc[split_idx:].copy()
        else:
            # Random split (not maintaining temporal order)
            train_data, test_data = train_test_split(df, test_size=test_size, random_state=random_state)

        return train_data, test_data

    def validate_features(self, df: pd.DataFrame) -> bool:
        """
        Validate that the dataframe contains all required features for the non-linear model.

        Args:
            df (pd.DataFrame): DataFrame to validate

        Returns:
            bool: True if all required features are present
        """
        required_cols = ['ozone', 'NO2', 'NOX', 'SO2', 'NMVOC', 'TEMP', 'RAD']
        missing_cols = [col for col in required_cols if col not in df.columns]

        if missing_cols:
            print(f"Missing required columns: {missing_cols}")
            return False

        return True

def dataloader():
    """
    Main function to demonstrate usage of the ExtendedNABELDataLoader class.
    """
    loader = ExtendedNABELDataLoader()

    try:
        # Load and prepare data
        print("Loading and preparing data...")
        data = loader.load_and_prepare_data()

        # Process data
        print("Processing data...")
        processed_data = loader.process_raw_data(data, handle_missing='interpolate')

        # Validate that we have all required features
        if not loader.validate_features(processed_data):
            raise ValueError("Dataset is missing required features for the non-linear model")

        # Split data
        print("Splitting data into train and test sets...")
        train_data, test_data = loader.split_data(processed_data, test_size=0.2, time_based=True)

        # Save processed data
        print("Saving processed data...")
        processed_dir = "data/processed"
        os.makedirs(processed_dir, exist_ok=True)

        train_data.to_csv(os.path.join(processed_dir, "train_data_full.csv"), index=False)
        test_data.to_csv(os.path.join(processed_dir, "test_data_full.csv"), index=False)

        print("\nData processing completed successfully!")
        print(f"Total samples: {len(processed_data)}")
        print(f"Training samples: {len(train_data)}")
        print(f"Testing samples: {len(test_data)}")

        # Print data summary
        print("\nData Summary:")
        print(processed_data.describe())

        # Print feature correlations with ozone
        print("\nFeature Correlations with Ozone:")
        correlations = processed_data.corr()['ozone'].sort_values(ascending=False)
        print(correlations)

    except Exception as e:
        print(f"Error: {e}")



dataloader()

Loading and preparing data...
Processing data...
Splitting data into train and test sets...
Saving processed data...

Data processing completed successfully!
Total samples: 2922
Training samples: 2337
Testing samples: 585

Data Summary:
                      date        ozone          NO2          NOX  \
count                 2922  2922.000000  2922.000000  2922.000000   
mean   2019-12-31 12:00:00    91.406742    23.685832    33.555065   
min    2016-01-01 00:00:00     3.600000     3.800000     4.100000   
25%    2017-12-31 06:00:00    60.325000    13.200000    14.900000   
50%    2019-12-31 12:00:00    88.700000    20.400000    24.300000   
75%    2021-12-30 18:00:00   122.175000    32.300000    44.400000   
max    2023-12-31 00:00:00   199.400000    79.800000   228.700000   
std                    NaN    42.961556    12.980111    26.133903   

               SO2        NMVOC         TEMP          RAD         Year  \
count  2922.000000  2922.000000  2922.000000  2922.000000  2922.000

In [ ]:
class OzoneBaselineModel:
    def __init__(self):
        """Initialize the baseline ozone prediction model."""
        self.model = LinearRegression()
        self.metrics = {}

    def train(self, X_train: np.ndarray, y_train: np.ndarray) -> None:
        """
        Train the linear regression model.

        Args:
            X_train (np.ndarray): Training features (radiation)
            y_train (np.ndarray): Training targets (ozone concentration)
        """
        self.model.fit(X_train.reshape(-1, 1), y_train)

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Make predictions using the trained model.

        Args:
            X (np.ndarray): Input features (radiation)

        Returns:
            np.ndarray: Predicted ozone concentrations
        """
        return self.model.predict(X.reshape(-1, 1))

    def evaluate(self, X_test: np.ndarray, y_test: np.ndarray) -> Dict[str, float]:
        """
        Evaluate the model performance.

        Args:
            X_test (np.ndarray): Test features
            y_test (np.ndarray): True test targets

        Returns:
            Dict[str, float]: Dictionary containing evaluation metrics
        """
        y_pred = self.predict(X_test)

        self.metrics = {
            'mae': mean_absolute_error(y_test, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_test, y_pred)),
            'r2': self.model.score(X_test.reshape(-1, 1), y_test),
            'coefficient': float(self.model.coef_[0]),
            'intercept': float(self.model.intercept_)
        }

        return self.metrics

    def plot_results(self, X_test: np.ndarray, y_test: np.ndarray,
                    dates_test: pd.DatetimeIndex = None,
                    save_dir: str = "results/figures") -> None:
        """
        Create and save various plots to analyze model performance.

        Args:
            X_test (np.ndarray): Test features
            y_test (np.ndarray): True test targets
            dates_test (pd.DatetimeIndex): Test dates for time series plot
            save_dir (str): Directory to save the plots
        """
        y_pred = self.predict(X_test)

        # Create figures directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)

        # 1. Scatter plot with regression line
        plt.figure(figsize=(10, 6))
        plt.scatter(X_test, y_test, alpha=0.5, label='Actual')
        plt.plot(X_test, y_pred, color='red', label='Predicted')
        plt.xlabel('Radiation (W/m²)')
        plt.ylabel('Ozone Concentration (µg/m³)')
        plt.title('Baseline Model: Ozone Concentration vs Radiation')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.savefig(os.path.join(save_dir, "scatter_plot.png"))
        plt.close()

        # 2. Residual plot
        residuals = y_test - y_pred
        plt.figure(figsize=(10, 6))
        plt.scatter(y_pred, residuals, alpha=0.5)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.xlabel('Predicted Ozone Concentration (µg/m³)')
        plt.ylabel('Residuals (µg/m³)')
        plt.title('Residual Plot')
        plt.grid(True, alpha=0.3)
        plt.savefig(os.path.join(save_dir, "residual_plot.png"))
        plt.close()

        # 3. Histogram of residuals
        plt.figure(figsize=(10, 6))
        sns.histplot(residuals, kde=True)
        plt.xlabel('Residuals (µg/m³)')
        plt.ylabel('Count')
        plt.title('Distribution of Residuals')
        plt.grid(True, alpha=0.3)
        plt.savefig(os.path.join(save_dir, "residual_distribution.png"))
        plt.close()

        # 4. Time series plot (if dates are provided)
        if dates_test is not None:
            plt.figure(figsize=(15, 6))
            plt.plot(dates_test, y_test, label='Actual', alpha=0.7)
            plt.plot(dates_test, y_pred, label='Predicted', alpha=0.7)
            plt.xlabel('Date')
            plt.ylabel('Ozone Concentration (µg/m³)')
            plt.title('Ozone Concentration Over Time')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(os.path.join(save_dir, "time_series.png"))
            plt.close()

    def save_metrics(self, save_path: str) -> None:
        """
        Save model metrics to a JSON file.

        Args:
            save_path (str): Path to save the metrics
        """
        with open(save_path, 'w') as f:
            json.dump(self.metrics, f, indent=4)

def lin_baseline():
    """Main function to train and evaluate the baseline model."""
    # Create results directory
    results_dir = "results"
    os.makedirs(results_dir, exist_ok=True)
    os.makedirs(os.path.join(results_dir, "figures"), exist_ok=True)
    os.makedirs(os.path.join(results_dir, "models"), exist_ok=True)

    # Load processed data
    processed_dir = "data/processed"
    train_data = pd.read_csv(os.path.join(processed_dir, "train_data.csv"))
    test_data = pd.read_csv(os.path.join(processed_dir, "test_data.csv"))

    # Convert date column to datetime
    train_data['date'] = pd.to_datetime(train_data['date'])
    test_data['date'] = pd.to_datetime(test_data['date'])

    # Extract features and targets
    X_train = train_data['radiation'].values
    y_train = train_data['ozone'].values
    X_test = test_data['radiation'].values
    y_test = test_data['ozone'].values

    # Initialize and train model
    model = OzoneBaselineModel()
    model.train(X_train, y_train)

    # Evaluate model
    metrics = model.evaluate(X_test, y_test)
    print("\nModel Evaluation Metrics:")
    print(f"MAE: {metrics['mae']:.2f} µg m⁻³")
    print(f"RMSE: {metrics['rmse']:.2f} µg m⁻³")
    print(f"R²: {metrics['r2']:.3f}")
    print(f"\nModel Equation:")
    print(f"[O₃] = {metrics['coefficient']:.3f} · Radiation + {metrics['intercept']:.3f} µg m⁻³")

    # Create visualizations
    model.plot_results(X_test, y_test, test_data['date'])

    # Save metrics
    model.save_metrics(os.path.join(results_dir, "models", "baseline_metrics.json"))



lin_baseline()



Model Evaluation Metrics:
MAE: 17.15 µg m⁻³
RMSE: 21.57 µg m⁻³
R²: 0.735

Model Equation:
[O₃] = 0.350 · Radiation + 33.962 µg m⁻³


## Non-linear Model

In [ ]:
"""
Non-linear regression model for ozone concentration prediction.
Based on Model 4 from the research paper using 12 key components of polynomial features.
"""

class OzoneNonLinearModel:
    def __init__(self, n_components: int = 12, degree: int = 3):
        """
        Initialize the non-linear ozone prediction model.

        Args:
            n_components (int): Number of components (terms) to include in the model
            degree (int): Degree of polynomial features to generate
        """
        self.n_components = n_components
        self.degree = degree
        self.base_features = ['NO2', 'NOX', 'SO2', 'NMVOC', 'TEMP', 'RAD']

        # Create polynomial features transformer (no column names to avoid warnings)
        self.poly = PolynomialFeatures(degree=self.degree, include_bias=True)

        # Create linear regression model
        self.model = LinearRegression()

        # Store metrics and important components
        self.metrics = {}
        self.selected_features = []
        self.feature_importances = {}
        self.selected_indices = None  # Will store indices of selected features

    def train(self, X_train: pd.DataFrame, y_train: np.ndarray) -> None:
        """
        Train the non-linear regression model.

        Args:
            X_train (pd.DataFrame): Training features (must include the 6 base features)
            y_train (np.ndarray): Training targets (ozone concentration)
        """
        # Check if all required features are present
        missing_features = [feat for feat in self.base_features if feat not in X_train.columns]
        if missing_features:
            raise ValueError(f"Missing required features: {missing_features}")

        # Extract features in the correct order
        X = X_train[self.base_features].values

        # Generate polynomial features
        X_poly = self.poly.fit_transform(X)

        # Get feature names
        self.feature_names = self.poly.get_feature_names_out(self.base_features)

        # Train full model first to determine important components
        self.full_model = LinearRegression().fit(X_poly, y_train)

        # Select the most important components
        self._select_important_components()

        # Use only selected components for final model
        selected_X_poly = self._get_selected_features(X_poly)

        # Train final model on selected components
        self.model.fit(selected_X_poly, y_train)

    def _select_important_components(self) -> None:
        """
        Select the most important components based on coefficient magnitude.
        """
        # Get absolute coefficients
        coefs = self.full_model.coef_
        abs_coefs = np.abs(coefs)

        # Find indices of the most important components (excluding intercept/bias)
        self.selected_indices = np.argsort(abs_coefs)[-self.n_components:]

        # Store selected feature names
        self.selected_features = [self.feature_names[i] for i in self.selected_indices]

        # Store feature importances with their names
        self.feature_importances = {
            self.feature_names[i]: coefs[i] for i in self.selected_indices
        }

    def _get_selected_features(self, X_poly: np.ndarray) -> np.ndarray:
        """
        Extract only the selected features from polynomial features.

        Args:
            X_poly (np.ndarray): Full polynomial features

        Returns:
            np.ndarray: Selected polynomial features
        """
        if self.selected_indices is None:
            raise ValueError("Model not trained yet. Call train() first.")

        # Extract only columns corresponding to selected indices
        return X_poly[:, self.selected_indices]

    def predict(self, X: pd.DataFrame) -> np.ndarray:
        """
        Make predictions using the trained model.

        Args:
            X (pd.DataFrame): Input features

        Returns:
            np.ndarray: Predicted ozone concentrations
        """
        # Check if model is trained
        if not hasattr(self, 'feature_names'):
            raise ValueError("Model not trained yet. Call train() first.")

        # Check if all required features are present
        missing_features = [feat for feat in self.base_features if feat not in X.columns]
        if missing_features:
            raise ValueError(f"Missing required features: {missing_features}")

        # Extract features in the correct order
        X_feat = X[self.base_features].values

        # Generate polynomial features
        X_poly = self.poly.transform(X_feat)

        # Extract only selected features
        selected_X_poly = self._get_selected_features(X_poly)

        # Make predictions
        return self.model.predict(selected_X_poly)

    def evaluate(self, X_test: pd.DataFrame, y_test: np.ndarray) -> Dict[str, float]:
        """
        Evaluate the model performance.

        Args:
            X_test (pd.DataFrame): Test features
            y_test (np.ndarray): True test targets

        Returns:
            Dict[str, float]: Dictionary containing evaluation metrics
        """
        y_pred = self.predict(X_test)

        # Extract features for R² calculation
        X_feat = X_test[self.base_features].values
        X_poly = self.poly.transform(X_feat)
        selected_X_poly = self._get_selected_features(X_poly)

        self.metrics = {
            'mae': mean_absolute_error(y_test, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_test, y_pred)),
            'mse': mean_squared_error(y_test, y_pred),
            'r2': self.model.score(selected_X_poly, y_test),
            'avg_predicted': float(np.mean(y_pred)),
            'avg_actual': float(np.mean(y_test)),
            'n_components': self.n_components
        }

        # Add coefficients to metrics
        self.metrics['intercept'] = float(self.model.intercept_)
        for i, idx in enumerate(self.selected_indices):
            feature_name = self.feature_names[idx]
            coef = self.model.coef_[i]
            self.metrics[f'component_{i+1}_name'] = feature_name
            self.metrics[f'component_{i+1}_coef'] = float(coef)

        return self.metrics

    def get_equation_string(self) -> str:
        """
        Get the prediction equation as a formatted string.

        Returns:
            str: Formatted equation string
        """
        if not hasattr(self, 'feature_names') or self.selected_indices is None:
            raise ValueError("Model not trained yet. Call train() first.")

        equation = "[O₃] = "

        # Add intercept
        equation += f"{self.model.intercept_:.1f}"

        # Add each component
        for i, idx in enumerate(self.selected_indices):
            feature = self.feature_names[idx]
            coef = self.model.coef_[i]

            # Format coefficient (handle very small or very large values)
            if abs(coef) < 0.001:
                coef_str = f"{coef:.1e}"
            elif abs(coef) < 0.01:
                coef_str = f"{coef:.4f}"
            elif abs(coef) < 0.1:
                coef_str = f"{coef:.3f}"
            elif abs(coef) < 1:
                coef_str = f"{coef:.2f}"
            else:
                coef_str = f"{coef:.1f}"

            # Add term with correct sign
            if coef >= 0:
                equation += f" + {coef_str} · {feature}"
            else:
                equation += f" - {abs(coef):.1f} · {feature}"

        # Add units
        equation += " μg m⁻³"

        return equation

    def plot_results(self, X_test: pd.DataFrame, y_test: np.ndarray,
                    dates_test: pd.DatetimeIndex = None,
                    save_dir: str = "results/figures/nonlinear") -> None:
        """
        Create and save various plots to analyze model performance.

        Args:
            X_test (pd.DataFrame): Test features
            y_test (np.ndarray): True test targets
            dates_test (pd.DatetimeIndex): Test dates for time series plot
            save_dir (str): Directory to save the plots
        """
        y_pred = self.predict(X_test)

        # Create figures directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)

        # 1. Actual vs Predicted scatter plot
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)

        # Add perfect prediction line
        max_val = max(np.max(y_test), np.max(y_pred))
        min_val = min(np.min(y_test), np.min(y_pred))
        plt.plot([min_val, max_val], [min_val, max_val], 'r--')

        plt.xlabel('Actual Ozone Concentration (µg/m³)')
        plt.ylabel('Predicted Ozone Concentration (µg/m³)')
        plt.title('Non-linear Model: Actual vs Predicted Ozone Concentration')
        plt.grid(True, alpha=0.3)

        # Add metrics text
        r2 = self.metrics.get('r2', 0)
        rmse = self.metrics.get('rmse', 0)
        mae = self.metrics.get('mae', 0)

        metrics_text = f"R² = {r2:.3f}\nRMSE = {rmse:.2f} µg/m³\nMAE = {mae:.2f} µg/m³"
        plt.annotate(metrics_text, xy=(0.05, 0.95), xycoords='axes fraction',
                     verticalalignment='top',
                     bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        plt.savefig(os.path.join(save_dir, "actual_vs_predicted.png"))
        plt.close()

        # 2. Residual plot
        residuals = y_test - y_pred
        plt.figure(figsize=(10, 6))
        plt.scatter(y_pred, residuals, alpha=0.5)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.xlabel('Predicted Ozone Concentration (µg/m³)')
        plt.ylabel('Residuals (µg/m³)')
        plt.title('Residual Plot')
        plt.grid(True, alpha=0.3)

        # Add residual statistics
        res_mean = np.mean(residuals)
        res_std = np.std(residuals)
        res_text = f"Mean = {res_mean:.2f} µg/m³\nStd Dev = {res_std:.2f} µg/m³"
        plt.annotate(res_text, xy=(0.05, 0.95), xycoords='axes fraction',
                     verticalalignment='top',
                     bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        plt.savefig(os.path.join(save_dir, "residual_plot.png"))
        plt.close()

        # 3. Histogram of residuals
        plt.figure(figsize=(10, 6))
        sns.histplot(residuals, kde=True)
        plt.xlabel('Residuals (µg/m³)')
        plt.ylabel('Count')
        plt.title('Distribution of Residuals')
        plt.grid(True, alpha=0.3)
        plt.savefig(os.path.join(save_dir, "residual_distribution.png"))
        plt.close()

        # 4. Time series plot (if dates are provided)
        if dates_test is not None:
            plt.figure(figsize=(15, 6))
            plt.plot(dates_test, y_test, label='Actual', alpha=0.7)
            plt.plot(dates_test, y_pred, label='Predicted', alpha=0.7)
            plt.xlabel('Date')
            plt.ylabel('Ozone Concentration (µg/m³)')
            plt.title('Ozone Concentration Over Time')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(os.path.join(save_dir, "time_series.png"))
            plt.close()

        # 5. Feature importance plot
        if self.feature_importances:
            # Convert to DataFrame for plotting
            importance_df = pd.DataFrame({
                'Feature': list(self.feature_importances.keys()),
                'Coefficient': list(self.feature_importances.values())
            })

            # Sort by absolute coefficient value
            importance_df['Abs_Coefficient'] = importance_df['Coefficient'].abs()
            importance_df = importance_df.sort_values('Abs_Coefficient', ascending=False)

            plt.figure(figsize=(12, 8))

            # Set colors based on coefficient sign (positive: blue, negative: red)
            colors = ['royalblue' if c > 0 else 'crimson' for c in importance_df['Coefficient']]

            sns.barplot(x='Coefficient', y='Feature', data=importance_df, palette=colors)
            plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
            plt.title('Feature Importance in Non-linear Model')
            plt.xlabel('Coefficient Value')
            plt.ylabel('Feature')
            plt.tight_layout()
            plt.savefig(os.path.join(save_dir, "feature_importance.png"))
            plt.close()

    def save_metrics(self, save_path: str) -> None:
        """
        Save model metrics to a JSON file.

        Args:
            save_path (str): Path to save the metrics
        """
        # Add equation string to metrics
        self.metrics['equation'] = self.get_equation_string()

        with open(save_path, 'w') as f:
            json.dump(self.metrics, f, indent=4)

    def save_model(self, save_path: str) -> None:
        """
        Save model details (not the full model) to a JSON file.

        Args:
            save_path (str): Path to save the model details
        """
        model_data = {
            'n_components': self.n_components,
            'degree': self.degree,
            'base_features': self.base_features,
            'selected_features': self.selected_features,
            'selected_indices': self.selected_indices.tolist(),
            'intercept': float(self.model.intercept_),
            'coefficients': [float(coef) for coef in self.model.coef_],
            'equation': self.get_equation_string(),
            'metrics': self.metrics
        }

        with open(save_path, 'w') as f:
            json.dump(model_data, f, indent=4)

def nonlinear_model():
    """Main function to train and evaluate the non-linear model."""
    # Create results directory
    results_dir = "results"
    model_dir = os.path.join(results_dir, "models")
    figures_dir = os.path.join(results_dir, "figures/nonlinear")
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(figures_dir, exist_ok=True)

    # Load processed data with all required features
    processed_dir = "data/processed"
    train_data = pd.read_csv(os.path.join(processed_dir, "train_data_full.csv"))
    test_data = pd.read_csv(os.path.join(processed_dir, "test_data_full.csv"))

    # Convert date column to datetime
    train_data['date'] = pd.to_datetime(train_data['date'])
    test_data['date'] = pd.to_datetime(test_data['date'])

    # Extract features and targets
    feature_cols = ['NO2', 'NOX', 'SO2', 'NMVOC', 'TEMP', 'RAD']
    X_train = train_data[feature_cols]
    y_train = train_data['ozone'].values
    X_test = test_data[feature_cols]
    y_test = test_data['ozone'].values

    # Initialize and train model
    model = OzoneNonLinearModel(n_components=12, degree=3)
    model.train(X_train, y_train)

    # Evaluate model
    metrics = model.evaluate(X_test, y_test)
    print("\nNon-linear Model Evaluation Metrics:")
    print(f"MAE: {metrics['mae']:.2f} µg m⁻³")
    print(f"RMSE: {metrics['rmse']:.2f} µg m⁻³")
    print(f"R²: {metrics['r2']:.3f}")

    print("\nModel Equation:")
    print(model.get_equation_string())

    # Print most important components
    print("\nMost Important Components:")
    for feature, coef in sorted(model.feature_importances.items(),
                              key=lambda x: abs(x[1]), reverse=True)[:12]:
        print(f"{feature}: {coef:.6f}")

    # Create visualizations
    model.plot_results(X_test, y_test, test_data['date'])

    # Save metrics and model
    model.save_metrics(os.path.join(model_dir, "nonlinear_metrics.json"))
    model.save_model(os.path.join(model_dir, "nonlinear_model.json"))


nonlinear_model()


Non-linear Model Evaluation Metrics:
MAE: 20.29 µg m⁻³
RMSE: 24.86 µg m⁻³
R²: 0.654

Model Equation:
[O₃] = 38.7 + 520.0 · SO2 NMVOC^2 - 11.7 · NOX NMVOC^2 + 3.4 · TEMP - 17.9 · SO2^2 NMVOC - 205.3 · NMVOC^2 TEMP + 44.0 · NMVOC TEMP - 5.5 · NO2 NMVOC^2 + 11.7 · SO2 - 14.6 · SO2 NMVOC + 962.5 · NMVOC^2 - 478.1 · NMVOC + 4094.6 · NMVOC^3 μg m⁻³

Most Important Components:
NMVOC^3: -1190.101631
NMVOC: 291.995193
NMVOC^2: -215.912660
SO2 NMVOC: -119.270920
SO2: -41.575159
NO2 NMVOC^2: 37.052094
NMVOC TEMP: -25.458000
NMVOC^2 TEMP: 23.750433
SO2^2 NMVOC: 18.168111
TEMP: -13.674769
NOX NMVOC^2: -10.935469
SO2 NMVOC^2: -10.553752


<ipython-input-20-4edf3391b82e>:319: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Coefficient', y='Feature', data=importance_df, palette=colors)


In [ ]:
"""
Non-linear regression model for ozone concentration prediction.
Based on Model 4 from the research paper, using the exact 12 components specified in the paper.
"""

class OzoneNonLinearModelExact:
    def __init__(self):
        """
        Initialize the non-linear ozone prediction model with the exact 12 components
        from the paper by Räss and Leuenberger.
        """
        self.base_features = ['NO2', 'NOX', 'SO2', 'NMVOC', 'TEMP', 'RAD']

        # Create polynomial features transformer with degree 3
        self.poly = PolynomialFeatures(degree=3, include_bias=True)

        # Create linear regression model
        self.model = LinearRegression()

        # Store metrics
        self.metrics = {}

        # The 12 components from the paper (will be set during training)
        self.component_indices = None
        self.feature_names = None

    def _find_component_indices(self):
        """
        Find the indices of the 12 specific components mentioned in the paper:

        1. NO2³
        2. NOX (linear term)
        3. NOX²
        4. NOX³
        5. T³ (temperature cubed)
        6. NO2 · T (interaction)
        7. NO2 · I (interaction with radiation)
        8. SO2 · NMVOC (interaction)
        9. SO2 · T (interaction)
        10. NMVOC · I (interaction)
        11. NOX · T (interaction)
        12. NOX · I (interaction)
        """
        # Define the 12 components we want to use (based on paper description)
        desired_components = [
            'NO2^3',            # NO2 cubed
            'NOX',              # NOX linear
            'NOX^2',            # NOX squared
            'NOX^3',            # NOX cubed
            'TEMP^3',           # Temperature cubed
            'NO2 TEMP',         # NO2 · T interaction
            'NO2 RAD',          # NO2 · I interaction
            'SO2 NMVOC',        # SO2 · NMVOC interaction
            'SO2 TEMP',         # SO2 · T interaction
            'NMVOC RAD',        # NMVOC · I interaction
            'NOX TEMP',         # NOX · T interaction
            'NOX RAD',          # NOX · I interaction
        ]

        # Maps for different possible naming formats
        component_formats = {
            'NO2^3': ['NO2^3', 'NO2 NO2 NO2', 'NO2**3', 'NO2³'],
            'NOX': ['NOX', 'NOX^1', 'NOX**1'],
            'NOX^2': ['NOX^2', 'NOX NOX', 'NOX**2', 'NOX²'],
            'NOX^3': ['NOX^3', 'NOX NOX NOX', 'NOX**3', 'NOX³'],
            'TEMP^3': ['TEMP^3', 'TEMP TEMP TEMP', 'TEMP**3', 'TEMP³'],
            'NO2 TEMP': ['NO2 TEMP', 'TEMP NO2'],
            'NO2 RAD': ['NO2 RAD', 'RAD NO2'],
            'SO2 NMVOC': ['SO2 NMVOC', 'NMVOC SO2'],
            'SO2 TEMP': ['SO2 TEMP', 'TEMP SO2'],
            'NMVOC RAD': ['NMVOC RAD', 'RAD NMVOC'],
            'NOX TEMP': ['NOX TEMP', 'TEMP NOX'],
            'NOX RAD': ['NOX RAD', 'RAD NOX']
        }

        # Find the indices of the desired components
        indices = []
        component_names = []

        # Create a lookup table for faster matching
        feature_dict = {name: idx for idx, name in enumerate(self.feature_names)}

        # First handle the bias/intercept term
        indices.append(0)  # Bias term is always at index 0
        component_names.append('1')  # Bias term is represented by 1

        # Find each of our desired components
        for component in desired_components:
            found = False
            # Try all possible naming formats
            for possible_name in component_formats[component]:
                # Check for exact match
                if possible_name in feature_dict:
                    indices.append(feature_dict[possible_name])
                    component_names.append(possible_name)
                    found = True
                    break

                # If not found, try more flexible matching for polynomial features
                if not found:
                    for feat_name in self.feature_names:
                        # For polynomial features, try matching individual terms
                        terms = possible_name.split()
                        if len(terms) > 1:
                            # Check if all terms appear in the feature name
                            if all(term in feat_name for term in terms):
                                indices.append(feature_dict[feat_name])
                                component_names.append(feat_name)
                                found = True
                                break

            if not found:
                # If we can't find the component, print a warning and continue
                print(f"Warning: Could not find component {component} in feature names")
                # As a fallback, let's use the most similar feature name
                best_match = None
                max_score = 0
                for name in self.feature_names:
                    # Simple similarity score: count matching words
                    score = sum(1 for term in component.split() if term in name.split())
                    if score > max_score:
                        max_score = score
                        best_match = name

                if best_match is not None:
                    indices.append(feature_dict[best_match])
                    component_names.append(best_match)
                    print(f"  Using {best_match} as a substitute")
                else:
                    # If we still can't find anything, use a dummy index
                    # This should not happen in practice
                    indices.append(1)  # Use first non-bias term as fallback
                    component_names.append(self.feature_names[1])
                    print(f"  Using {self.feature_names[1]} as a fallback")

        # Store the indices and names
        self.component_indices = indices
        self.selected_features = component_names

        print(f"Selected components:")
        for i, (idx, name) in enumerate(zip(indices, component_names)):
            print(f"{i}: {name} (index {idx})")

    def train(self, X_train: pd.DataFrame, y_train: np.ndarray) -> None:
        """
        Train the non-linear regression model.

        Args:
            X_train (pd.DataFrame): Training features (must include the 6 base features)
            y_train (np.ndarray): Training targets (ozone concentration)
        """
        # Check if all required features are present
        missing_features = [feat for feat in self.base_features if feat not in X_train.columns]
        if missing_features:
            raise ValueError(f"Missing required features: {missing_features}")

        # Extract features in the correct order
        X = X_train[self.base_features].values

        # Generate polynomial features
        X_poly = self.poly.fit_transform(X)

        # Get feature names
        self.feature_names = self.poly.get_feature_names_out(self.base_features)

        # Find the indices of the 12 specific components from the paper
        self._find_component_indices()

        # Select only the components we want
        X_selected = X_poly[:, self.component_indices]

        # Train the model on selected components
        self.model.fit(X_selected, y_train)

        # Store the coefficients for the components
        self.feature_importances = {}
        for i, idx in enumerate(self.component_indices):
            if i == 0:  # Skip the bias term
                continue

            feature_name = self.selected_features[i]
            coef = self.model.coef_[i]
            self.feature_importances[feature_name] = coef

    def predict(self, X: pd.DataFrame) -> np.ndarray:
        """
        Make predictions using the trained model.

        Args:
            X (pd.DataFrame): Input features

        Returns:
            np.ndarray: Predicted ozone concentrations
        """
        # Check if model is trained
        if not hasattr(self, 'feature_names'):
            raise ValueError("Model not trained yet. Call train() first.")

        # Check if all required features are present
        missing_features = [feat for feat in self.base_features if feat not in X.columns]
        if missing_features:
            raise ValueError(f"Missing required features: {missing_features}")

        # Extract features in the correct order
        X_feat = X[self.base_features].values

        # Generate polynomial features
        X_poly = self.poly.transform(X_feat)

        # Extract only selected components
        X_selected = X_poly[:, self.component_indices]

        # Make predictions
        return self.model.predict(X_selected)

    def evaluate(self, X_test: pd.DataFrame, y_test: np.ndarray) -> Dict[str, float]:
        """
        Evaluate the model performance.

        Args:
            X_test (pd.DataFrame): Test features
            y_test (np.ndarray): True test targets

        Returns:
            Dict[str, float]: Dictionary containing evaluation metrics
        """
        y_pred = self.predict(X_test)

        # Extract features for R² calculation
        X_feat = X_test[self.base_features].values
        X_poly = self.poly.transform(X_feat)
        X_selected = X_poly[:, self.component_indices]

        self.metrics = {
            'mae': mean_absolute_error(y_test, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_test, y_pred)),
            'mse': mean_squared_error(y_test, y_pred),
            'r2': self.model.score(X_selected, y_test),
            'avg_predicted': float(np.mean(y_pred)),
            'avg_actual': float(np.mean(y_test)),
            'n_components': len(self.component_indices) - 1  # Subtract 1 for bias term
        }

        # Add coefficients to metrics
        self.metrics['intercept'] = float(self.model.intercept_)
        for i, idx in enumerate(self.component_indices):
            if i == 0:  # Skip the bias term
                continue

            feature_name = self.selected_features[i]
            coef = self.model.coef_[i-1]  # -1 because coef doesn't include intercept
            self.metrics[f'component_{i}_name'] = feature_name
            self.metrics[f'component_{i}_coef'] = float(coef)

        return self.metrics

    def get_equation_string(self) -> str:
        """
        Get the prediction equation as a formatted string.

        Returns:
            str: Formatted equation string
        """
        if not hasattr(self, 'feature_names') or self.component_indices is None:
            raise ValueError("Model not trained yet. Call train() first.")

        equation = "[O₃] = "

        # Add intercept
        equation += f"{self.model.intercept_:.1f}"

        # Add each component
        for i, idx in enumerate(self.component_indices[1:], 1):  # Start at 1 to skip bias
            feature = self.selected_features[i]
            coef = self.model.coef_[i-1]  # -1 because coef_ doesn't include intercept

            # Format coefficient (handle very small or very large values)
            if abs(coef) < 0.001:
                coef_str = f"{coef:.1e}"
            elif abs(coef) < 0.01:
                coef_str = f"{coef:.4f}"
            elif abs(coef) < 0.1:
                coef_str = f"{coef:.3f}"
            elif abs(coef) < 1:
                coef_str = f"{coef:.2f}"
            else:
                coef_str = f"{coef:.1f}"

            # Add term with correct sign
            if coef >= 0:
                equation += f" + {coef_str} · {feature}"
            else:
                equation += f" - {abs(coef):.1f} · {feature}"

        # Add units
        equation += " μg m⁻³"

        return equation

    def plot_results(self, X_test: pd.DataFrame, y_test: np.ndarray,
                    dates_test: pd.DatetimeIndex = None,
                    save_dir: str = "results/figures/nonlinear_exact") -> None:
        """
        Create and save various plots to analyze model performance.

        Args:
            X_test (pd.DataFrame): Test features
            y_test (np.ndarray): True test targets
            dates_test (pd.DatetimeIndex): Test dates for time series plot
            save_dir (str): Directory to save the plots
        """
        y_pred = self.predict(X_test)

        # Create figures directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)

        # 1. Actual vs Predicted scatter plot
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)

        # Add perfect prediction line
        max_val = max(np.max(y_test), np.max(y_pred))
        min_val = min(np.min(y_test), np.min(y_pred))
        plt.plot([min_val, max_val], [min_val, max_val], 'r--')

        plt.xlabel('Actual Ozone Concentration (µg/m³)')
        plt.ylabel('Predicted Ozone Concentration (µg/m³)')
        plt.title('Non-linear Model (Paper Components): Actual vs Predicted Ozone')
        plt.grid(True, alpha=0.3)

        # Add metrics text
        r2 = self.metrics.get('r2', 0)
        rmse = self.metrics.get('rmse', 0)
        mae = self.metrics.get('mae', 0)

        metrics_text = f"R² = {r2:.3f}\nRMSE = {rmse:.2f} µg/m³\nMAE = {mae:.2f} µg/m³"
        plt.annotate(metrics_text, xy=(0.05, 0.95), xycoords='axes fraction',
                     verticalalignment='top',
                     bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        plt.savefig(os.path.join(save_dir, "actual_vs_predicted.png"))
        plt.close()

        # 2. Residual plot
        residuals = y_test - y_pred
        plt.figure(figsize=(10, 6))
        plt.scatter(y_pred, residuals, alpha=0.5)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.xlabel('Predicted Ozone Concentration (µg/m³)')
        plt.ylabel('Residuals (µg/m³)')
        plt.title('Residual Plot')
        plt.grid(True, alpha=0.3)

        # Add residual statistics
        res_mean = np.mean(residuals)
        res_std = np.std(residuals)
        res_text = f"Mean = {res_mean:.2f} µg/m³\nStd Dev = {res_std:.2f} µg/m³"
        plt.annotate(res_text, xy=(0.05, 0.95), xycoords='axes fraction',
                     verticalalignment='top',
                     bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        plt.savefig(os.path.join(save_dir, "residual_plot.png"))
        plt.close()

        # 3. Histogram of residuals
        plt.figure(figsize=(10, 6))
        sns.histplot(residuals, kde=True)
        plt.xlabel('Residuals (µg/m³)')
        plt.ylabel('Count')
        plt.title('Distribution of Residuals')
        plt.grid(True, alpha=0.3)
        plt.savefig(os.path.join(save_dir, "residual_distribution.png"))
        plt.close()

        # 4. Time series plot (if dates are provided)
        if dates_test is not None:
            plt.figure(figsize=(15, 6))
            plt.plot(dates_test, y_test, label='Actual', alpha=0.7)
            plt.plot(dates_test, y_pred, label='Predicted', alpha=0.7)
            plt.xlabel('Date')
            plt.ylabel('Ozone Concentration (µg/m³)')
            plt.title('Ozone Concentration Over Time')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(os.path.join(save_dir, "time_series.png"))
            plt.close()

        # 5. Feature importance plot
        if hasattr(self, 'feature_importances') and self.feature_importances:
            # Convert to DataFrame for plotting
            importance_df = pd.DataFrame({
                'Feature': list(self.feature_importances.keys()),
                'Coefficient': list(self.feature_importances.values())
            })

            # Sort by absolute coefficient value
            importance_df['Abs_Coefficient'] = importance_df['Coefficient'].abs()
            importance_df = importance_df.sort_values('Abs_Coefficient', ascending=False)

            plt.figure(figsize=(12, 8))

            # Create a color mapping for positive/negative values
            importance_df['sign'] = np.where(importance_df['Coefficient'] >= 0, 'Positive', 'Negative')

            # Create barplot with hue parameter
            sns.barplot(
                x='Coefficient',
                y='Feature',
                data=importance_df,
                hue='sign',  # Use the sign column for coloring
                palette={'Positive': 'royalblue', 'Negative': 'crimson'},
                legend=False  # Don't show the legend
            )

            plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
            plt.title('Feature Importance in Non-linear Model (Paper Components)')
            plt.xlabel('Coefficient Value')
            plt.ylabel('Feature')
            plt.tight_layout()
            plt.savefig(os.path.join(save_dir, "feature_importance.png"))
            plt.close()

    def save_metrics(self, save_path: str) -> None:
        """
        Save model metrics to a JSON file.

        Args:
            save_path (str): Path to save the metrics
        """
        # Add equation string to metrics
        self.metrics['equation'] = self.get_equation_string()

        with open(save_path, 'w') as f:
            json.dump(self.metrics, f, indent=4)

    def save_model(self, save_path: str) -> None:
        """
        Save model details (not the full model) to a JSON file.

        Args:
            save_path (str): Path to save the model details
        """
        model_data = {
            'base_features': self.base_features,
            'selected_features': self.selected_features,
            'component_indices': self.component_indices,
            'intercept': float(self.model.intercept_),
            'coefficients': [float(coef) for coef in self.model.coef_],
            'equation': self.get_equation_string(),
            'metrics': self.metrics
        }

        with open(save_path, 'w') as f:
            json.dump(model_data, f, indent=4)


"""
Function to display model results in a more visually appealing format.
"""

def display_pretty_results(model, metrics, display_components=True):
    """
    Display model results in a nicely formatted way.

    Args:
        model: The trained model
        metrics: Dictionary containing model evaluation metrics
        display_components: Whether to display component coefficients
    """
    # Terminal formatting codes
    BOLD = '\033[1m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    ENDC = '\033[0m'

    # Print title
    print(f"\n{BOLD}{BLUE}═══════════════════════════════════════════════════════{ENDC}")
    print(f"{BOLD}{BLUE}           NON-LINEAR OZONE PREDICTION MODEL          {ENDC}")
    print(f"{BOLD}{BLUE}═══════════════════════════════════════════════════════{ENDC}\n")

    # Print model information
    print(f"{BOLD}Model Information:{ENDC}")
    print(f"  Type: Non-linear Regression with Paper-Specified Components")
    print(f"  Base Features: {', '.join(model.base_features)}")
    print(f"  Polynomial Degree: 3")
    print(f"  Number of Components: {metrics['n_components']}")

    # Print key metrics
    print(f"\n{BOLD}Performance Metrics:{ENDC}")
    print(f"  {BOLD}MAE:{ENDC}     {GREEN}{metrics['mae']:.2f} µg/m³{ENDC}")
    print(f"  {BOLD}RMSE:{ENDC}    {GREEN}{metrics['rmse']:.2f} µg/m³{ENDC}")
    print(f"  {BOLD}R²:{ENDC}      {GREEN}{metrics['r2']:.4f}{ENDC}")
    print(f"  {BOLD}MSE:{ENDC}     {metrics['mse']:.2f}")

    # Print prediction averages
    print(f"\n{BOLD}Prediction Summary:{ENDC}")
    print(f"  Average Predicted: {metrics['avg_predicted']:.2f} µg/m³")
    print(f"  Average Actual:    {metrics['avg_actual']:.2f} µg/m³")
    print(f"  Difference:        {abs(metrics['avg_predicted'] - metrics['avg_actual']):.2f} µg/m³")

    # Print model equation
    print(f"\n{BOLD}Model Equation:{ENDC}")
    # Split equation into multiple lines for readability
    equation = metrics.get('equation', model.get_equation_string())
    equation_parts = equation.split('+')
    formatted_equation = equation_parts[0].strip()  # Start with the first part (usually intercept)
    line_length = len(formatted_equation)

    for part in equation_parts[1:]:
        if '·' in part:  # This is a term
            term = part.strip()
            if line_length + len(term) + 3 > 80:  # Start new line if too long
                formatted_equation += f"\n  + {term}"
                line_length = 4 + len(term)
            else:
                formatted_equation += f" + {term}"
                line_length += 3 + len(term)

    # Handle negative terms
    formatted_equation = formatted_equation.replace("+ -", "- ")

    # Print the formatted equation
    print(f"  {formatted_equation}")

    # Print component coefficients
    if display_components:
        print(f"\n{BOLD}Component Coefficients:{ENDC}")
        print(f"  {BOLD}{'Component':<25} {'Coefficient':>15}{ENDC}")
        print(f"  {'-'*42}")

        # Print intercept
        print(f"  {'Intercept':<25} {metrics['intercept']:>15.4f}")

        # Print all other components
        for i in range(1, metrics['n_components'] + 1):
            component_name = metrics.get(f'component_{i}_name', f'Component {i}')
            component_coef = metrics.get(f'component_{i}_coef', 0.0)

            # Color based on coefficient value
            if abs(component_coef) < 0.001:
                coef_str = f"{YELLOW}{component_coef:>15.6f}{ENDC}"
            elif component_coef >= 0:
                coef_str = f"{GREEN}{component_coef:>15.4f}{ENDC}"
            else:
                coef_str = f"{RED}{component_coef:>15.4f}{ENDC}"

            print(f"  {component_name:<25} {coef_str}")

    # Print footer
    print(f"\n{BOLD}{BLUE}═══════════════════════════════════════════════════════{ENDC}")
    print(f"{BOLD}{BLUE}                      END OF REPORT                    {ENDC}")
    print(f"{BOLD}{BLUE}═══════════════════════════════════════════════════════{ENDC}\n")



def nonlinear_model_paper_components():
    """Main function to train and evaluate the non-linear model with paper components."""
    # Create results directory
    results_dir = "results"
    model_dir = os.path.join(results_dir, "models")
    figures_dir = os.path.join(results_dir, "figures/nonlinear_exact")
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(figures_dir, exist_ok=True)

    # Load processed data with all required features
    processed_dir = "data/processed"
    train_data = pd.read_csv(os.path.join(processed_dir, "train_data_full.csv"))
    test_data = pd.read_csv(os.path.join(processed_dir, "test_data_full.csv"))

    # Convert date column to datetime
    train_data['date'] = pd.to_datetime(train_data['date'])
    test_data['date'] = pd.to_datetime(test_data['date'])

    # Extract features and targets
    feature_cols = ['NO2', 'NOX', 'SO2', 'NMVOC', 'TEMP', 'RAD']
    X_train = train_data[feature_cols]
    y_train = train_data['ozone'].values
    X_test = test_data[feature_cols]
    y_test = test_data['ozone'].values

    # Print basic data statistics
    print("\nData Summary:")
    print(f"Training samples: {len(X_train)}")
    print(f"Testing samples: {len(X_test)}")
    print(f"Training date range: {train_data['date'].min()} to {train_data['date'].max()}")
    print(f"Testing date range: {test_data['date'].min()} to {test_data['date'].max()}")

    # Print feature statistics
    print("\nFeature statistics (training data):")
    print(X_train.describe())

    # Print ozone statistics
    print("\nOzone statistics:")
    print(f"Training mean: {np.mean(y_train):.2f} μg/m³")
    print(f"Training std: {np.std(y_train):.2f} μg/m³")
    print(f"Test mean: {np.mean(y_test):.2f} μg/m³")
    print(f"Test std: {np.std(y_test):.2f} μg/m³")

    # Initialize and train model
    model = OzoneNonLinearModelExact()
    print("\nTraining model with paper-specified components...")
    model.train(X_train, y_train)

    # Evaluate model
    print("\nEvaluating model...")
    metrics = model.evaluate(X_test, y_test)
    print("\nNon-linear Model Evaluation Metrics (Paper Components):")
    print(f"MAE: {metrics['mae']:.2f} µg m⁻³")
    print(f"RMSE: {metrics['rmse']:.2f} µg m⁻³")
    print(f"R²: {metrics['r2']:.3f}")

    print("\nModel Equation:")
    print(model.get_equation_string())

    # Print component coefficients
    print("\nComponent Coefficients:")
    for i, idx in enumerate(model.component_indices[1:], 1):  # Skip bias term
        feature_name = model.selected_features[i]
        coef = model.model.coef_[i-1]  # -1 because coef doesn't include intercept
        print(f"{feature_name}: {coef:.6f}")

    # Create visualizations
    print("\nCreating visualization plots...")
    model.plot_results(X_test, y_test, test_data['date'])

    # Save metrics and model
    model.save_metrics(os.path.join(model_dir, "nonlinear_exact_metrics.json"))
    model.save_model(os.path.join(model_dir, "nonlinear_exact_model.json"))

    print("\nModel training and evaluation complete!")

    display_pretty_results(model, metrics)

nonlinear_model_paper_components()


Data Summary:
Training samples: 2337
Testing samples: 585
Training date range: 2016-01-01 00:00:00 to 2022-05-25 00:00:00
Testing date range: 2022-05-26 00:00:00 to 2023-12-31 00:00:00

Feature statistics (training data):
               NO2          NOX          SO2        NMVOC         TEMP  \
count  2337.000000  2337.000000  2337.000000  2337.000000  2337.000000   
mean     24.903252    35.485216     1.656739     0.105370    13.349807   
std      13.335278    27.443950     1.319634     0.058972     7.041906   
min       3.800000     4.100000     0.100000     0.000000    -2.800000   
25%      13.900000    16.000000     0.600000     0.100000     7.200000   
50%      21.800000    26.000000     1.200000     0.100000    12.800000   
75%      34.200000    47.500000     2.400000     0.100000    19.700000   
max      79.800000   228.700000     6.500000     0.400000    28.800000   

               RAD  
count  2337.000000  
mean    161.919042  
std     100.255408  
min       1.900000  
25%  

In [ ]:
"""
Recurrent Neural Network model for ozone concentration prediction.
Based on Model 8/9 from the research paper.
"""

# Suppress TensorFlow warnings
tf.get_logger().setLevel('ERROR')

class OzoneRNNModel:
    def __init__(self, rnn_type: str = 'lstm', n_features: int = 12,
                 sequence_length: int = 1, name: str = "Model8_RNN"):
        """
        Initialize the RNN model for ozone prediction.

        Args:
            rnn_type (str): Type of RNN to use ('lstm', 'gru', or 'simple')
            n_features (int): Number of input features
            sequence_length (int): Length of input sequences for temporal modeling
            name (str): Name of the model
        """
        self.rnn_type = rnn_type.lower()
        self.n_features = n_features
        self.sequence_length = sequence_length
        self.name = name

        # Initialize model
        self.model = None

        # Initialize scalers
        self.feature_scaler = StandardScaler()
        self.target_scaler = StandardScaler()

        # Store metrics
        self.metrics = {}
        self.history = None

        # Feature names (will be set during training)
        self.feature_names = []

    def _build_model(self) -> None:
        """
        Build the RNN model architecture.
        """
        # Clear previous session to avoid conflicts
        tf.keras.backend.clear_session()

        # Define input shape based on sequence length and number of features
        input_shape = (self.sequence_length, self.n_features)

        # Create sequential model
        model = Sequential(name=self.name)

        # Add RNN layer based on the specified type
        if self.rnn_type == 'lstm':
            model.add(LSTM(64, input_shape=input_shape, return_sequences=True,
                          activation='tanh', name='lstm_1'))
            model.add(Dropout(0.2))
            model.add(LSTM(32, activation='tanh', name='lstm_2'))
        elif self.rnn_type == 'gru':
            model.add(GRU(64, input_shape=input_shape, return_sequences=True,
                         activation='tanh', name='gru_1'))
            model.add(Dropout(0.2))
            model.add(GRU(32, activation='tanh', name='gru_2'))
        else:  # simple RNN
            model.add(SimpleRNN(64, input_shape=input_shape, return_sequences=True,
                              activation='tanh', name='rnn_1'))
            model.add(Dropout(0.2))
            model.add(SimpleRNN(32, activation='tanh', name='rnn_2'))

        # Add output Dense layers
        model.add(Dense(16, activation='relu', name='dense_1'))
        model.add(Dense(1, activation='linear', name='output'))

        # Compile model
        model.compile(optimizer=Adam(learning_rate=0.001),
                      loss='mse',
                      metrics=['mae'])

        # Store model
        self.model = model

    def _prepare_sequences(self, X: np.ndarray) -> np.ndarray:
        """
        Prepare input sequences for RNN.

        Args:
            X (np.ndarray): Input features

        Returns:
            np.ndarray: Prepared sequences
        """
        # If sequence length is 1, reshape to (samples, 1, features)
        if self.sequence_length == 1:
            return X.reshape((X.shape[0], self.sequence_length, X.shape[1]))

        # Otherwise, create proper sequences
        sequences = []
        for i in range(len(X) - self.sequence_length + 1):
            sequences.append(X[i:i + self.sequence_length])

        return np.array(sequences)

    def train(self, X_train: pd.DataFrame, y_train: np.ndarray,
              validation_split: float = 0.2, epochs: int = 100,
              batch_size: int = 32, patience: int = 20) -> None:
        """
        Train the RNN model.

        Args:
            X_train (pd.DataFrame): Training features
            y_train (np.ndarray): Training targets (ozone concentration)
            validation_split (float): Fraction of training data to use for validation
            epochs (int): Maximum number of training epochs
            batch_size (int): Batch size for training
            patience (int): Patience for early stopping
        """
        # Store feature names
        self.feature_names = X_train.columns.tolist()
        self.n_features = len(self.feature_names)

        # Scale features
        X_scaled = self.feature_scaler.fit_transform(X_train)

        # Reshape y_train for scaler
        y_reshaped = y_train.reshape(-1, 1)
        y_scaled = self.target_scaler.fit_transform(y_reshaped).flatten()

        # Prepare input sequences
        X_sequences = self._prepare_sequences(X_scaled)

        # Build the model
        self._build_model()

        # Print model summary
        self.model.summary()

        # Set up callbacks
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)
        ]

        # Train the model
        self.history = self.model.fit(
            X_sequences, y_scaled,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=validation_split,
            callbacks=callbacks,
            verbose=1
        )

    def predict(self, X: pd.DataFrame) -> np.ndarray:
        """
        Make predictions using the trained model.

        Args:
            X (pd.DataFrame): Input features

        Returns:
            np.ndarray: Predicted ozone concentrations
        """
        # Check if model is built
        if self.model is None:
            raise ValueError("Model not trained yet. Call train() first.")

        # Check feature order
        missing_features = [feat for feat in self.feature_names if feat not in X.columns]
        if missing_features:
            raise ValueError(f"Missing features: {missing_features}")

        # Ensure correct feature order
        X = X[self.feature_names]

        # Scale features
        X_scaled = self.feature_scaler.transform(X)

        # Prepare sequences
        X_sequences = self._prepare_sequences(X_scaled)

        # Make predictions
        y_pred_scaled = self.model.predict(X_sequences)

        # Inverse transform to get original scale
        y_pred = self.target_scaler.inverse_transform(y_pred_scaled).flatten()

        return y_pred

    def evaluate(self, X_test: pd.DataFrame, y_test: np.ndarray) -> Dict[str, float]:
        """
        Evaluate the model on test data.

        Args:
            X_test (pd.DataFrame): Test features
            y_test (np.ndarray): True test targets

        Returns:
            Dict[str, float]: Dictionary containing evaluation metrics
        """
        # Make predictions
        y_pred = self.predict(X_test)

        # Calculate metrics
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)

        # Store metrics
        self.metrics = {
            'mae': float(mae),
            'mse': float(mse),
            'rmse': float(rmse),
            'r2': float(r2),
            'avg_predicted': float(np.mean(y_pred)),
            'avg_actual': float(np.mean(y_test)),
            'rnn_type': self.rnn_type,
            'sequence_length': self.sequence_length,
            'n_features': self.n_features
        }

        return self.metrics

    def plot_results(self, X_test: pd.DataFrame, y_test: np.ndarray,
                    dates_test: pd.DatetimeIndex = None,
                    save_dir: str = "results/figures/rnn") -> None:
        """
        Create and save plots to visualize model performance.

        Args:
            X_test (pd.DataFrame): Test features
            y_test (np.ndarray): True test targets
            dates_test (pd.DatetimeIndex): Test dates for time series plot
            save_dir (str): Directory to save the plots
        """
        # Create save directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)

        # Make predictions
        y_pred = self.predict(X_test)

        # 1. Training history plot
        if self.history is not None:
            plt.figure(figsize=(12, 5))

            # Plot training & validation loss
            plt.subplot(1, 2, 1)
            plt.plot(self.history.history['loss'], label='Training Loss')
            plt.plot(self.history.history['val_loss'], label='Validation Loss')
            plt.title('Model Loss During Training')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            plt.grid(alpha=0.3)

            # Plot training & validation MAE
            plt.subplot(1, 2, 2)
            plt.plot(self.history.history['mae'], label='Training MAE')
            plt.plot(self.history.history['val_mae'], label='Validation MAE')
            plt.title('Model MAE During Training')
            plt.xlabel('Epoch')
            plt.ylabel('MAE')
            plt.legend()
            plt.grid(alpha=0.3)

            plt.tight_layout()
            plt.savefig(os.path.join(save_dir, "training_history.png"))
            plt.close()

        # 2. Actual vs Predicted scatter plot
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)

        # Add perfect prediction line
        max_val = max(np.max(y_test), np.max(y_pred))
        min_val = min(np.min(y_test), np.min(y_pred))
        plt.plot([min_val, max_val], [min_val, max_val], 'r--')

        plt.xlabel('Actual Ozone Concentration (µg/m³)')
        plt.ylabel('Predicted Ozone Concentration (µg/m³)')
        plt.title(f'{self.rnn_type.upper()} Model: Actual vs Predicted Ozone')
        plt.grid(True, alpha=0.3)

        # Add metrics text
        r2 = self.metrics.get('r2', 0)
        rmse = self.metrics.get('rmse', 0)
        mae = self.metrics.get('mae', 0)

        metrics_text = f"R² = {r2:.3f}\nRMSE = {rmse:.2f} µg/m³\nMAE = {mae:.2f} µg/m³"
        plt.annotate(metrics_text, xy=(0.05, 0.95), xycoords='axes fraction',
                     verticalalignment='top',
                     bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        plt.savefig(os.path.join(save_dir, "actual_vs_predicted.png"))
        plt.close()

        # 3. Residual plot
        residuals = y_test - y_pred
        plt.figure(figsize=(10, 6))
        plt.scatter(y_pred, residuals, alpha=0.5)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.xlabel('Predicted Ozone Concentration (µg/m³)')
        plt.ylabel('Residuals (µg/m³)')
        plt.title('Residual Plot')
        plt.grid(True, alpha=0.3)

        # Add residual statistics
        res_mean = np.mean(residuals)
        res_std = np.std(residuals)
        res_text = f"Mean = {res_mean:.2f} µg/m³\nStd Dev = {res_std:.2f} µg/m³"
        plt.annotate(res_text, xy=(0.05, 0.95), xycoords='axes fraction',
                     verticalalignment='top',
                     bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        plt.savefig(os.path.join(save_dir, "residual_plot.png"))
        plt.close()

        # 4. Time series plot (if dates are provided)
        if dates_test is not None:
            plt.figure(figsize=(15, 6))
            plt.plot(dates_test, y_test, label='Actual', alpha=0.7)
            plt.plot(dates_test, y_pred, label='Predicted', alpha=0.7)
            plt.xlabel('Date')
            plt.ylabel('Ozone Concentration (µg/m³)')
            plt.title('Ozone Concentration Over Time')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(os.path.join(save_dir, "time_series.png"))
            plt.close()

            # 5. Residuals over time
            plt.figure(figsize=(15, 6))
            plt.plot(dates_test, residuals, color='royalblue', alpha=0.7)
            plt.axhline(y=0, color='r', linestyle='--')
            plt.xlabel('Date')
            plt.ylabel('Residuals (µg/m³)')
            plt.title('Prediction Residuals Over Time')
            plt.grid(True, alpha=0.3)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(os.path.join(save_dir, "residuals_time_series.png"))
            plt.close()

        # 6. Histogram of residuals
        plt.figure(figsize=(10, 6))
        sns.histplot(residuals, kde=True)
        plt.xlabel('Residuals (µg/m³)')
        plt.ylabel('Count')
        plt.title('Distribution of Residuals')
        plt.grid(True, alpha=0.3)
        plt.savefig(os.path.join(save_dir, "residual_distribution.png"))
        plt.close()

    def save_metrics(self, save_path: str) -> None:
        """
        Save model metrics to a JSON file.

        Args:
            save_path (str): Path to save the metrics
        """
        with open(save_path, 'w') as f:
            json.dump(self.metrics, f, indent=4)

    def save_model(self, save_dir: str) -> None:
        """
        Save the trained model.

        Args:
            save_dir (str): Directory to save the model
        """
        # Create directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)

        # Create model name
        model_path = os.path.join(save_dir, f"{self.name}.h5")

        # Save model
        self.model.save(model_path)

        # Save scalers
        import pickle
        with open(os.path.join(save_dir, f"{self.name}_feature_scaler.pkl"), 'wb') as f:
            pickle.dump(self.feature_scaler, f)

        with open(os.path.join(save_dir, f"{self.name}_target_scaler.pkl"), 'wb') as f:
            pickle.dump(self.target_scaler, f)

        print(f"Model saved to {model_path}")

    @classmethod
    def load_model(cls, model_dir: str, model_name: str) -> 'OzoneRNNModel':
        """
        Load a saved model.

        Args:
            model_dir (str): Directory where the model is saved
            model_name (str): Name of the model

        Returns:
            OzoneRNNModel: Loaded model
        """
        # Create model path
        model_path = os.path.join(model_dir, f"{model_name}.h5")

        # Load model
        loaded_model = tf.keras.models.load_model(model_path)

        # Load scalers
        import pickle
        with open(os.path.join(model_dir, f"{model_name}_feature_scaler.pkl"), 'rb') as f:
            feature_scaler = pickle.load(f)

        with open(os.path.join(model_dir, f"{model_name}_target_scaler.pkl"), 'rb') as f:
            target_scaler = pickle.load(f)

        # Create new instance
        instance = cls(name=model_name)
        instance.model = loaded_model
        instance.feature_scaler = feature_scaler
        instance.target_scaler = target_scaler

        # Try to determine RNN type from model layers
        for layer in loaded_model.layers:
            if 'lstm' in layer.name:
                instance.rnn_type = 'lstm'
                break
            elif 'gru' in layer.name:
                instance.rnn_type = 'gru'
                break
            elif 'rnn' in layer.name:
                instance.rnn_type = 'simple'
                break

        return instance

def display_pretty_results(model, metrics, display_components=True):
    """
    Display model results in a nicely formatted way.

    Args:
        model: The trained model
        metrics: Dictionary containing model evaluation metrics
        display_components: Whether to display component coefficients
    """
    # Terminal formatting codes
    BOLD = '\033[1m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    ENDC = '\033[0m'

    # Get RNN type
    rnn_type = metrics.get('rnn_type', 'RNN').upper()

    # Print title
    print(f"\n{BOLD}{BLUE}═══════════════════════════════════════════════════════{ENDC}")
    print(f"{BOLD}{BLUE}           {rnn_type} OZONE PREDICTION MODEL            {ENDC}")
    print(f"{BOLD}{BLUE}═══════════════════════════════════════════════════════{ENDC}\n")

    # Print model information
    print(f"{BOLD}Model Information:{ENDC}")
    print(f"  Type: {rnn_type} Neural Network")
    print(f"  Features: {metrics.get('n_features', 'Unknown')}")

    if 'sequence_length' in metrics:
        print(f"  Sequence Length: {metrics['sequence_length']}")

    # Print key metrics
    print(f"\n{BOLD}Performance Metrics:{ENDC}")
    print(f"  {BOLD}MAE:{ENDC}     {GREEN}{metrics['mae']:.2f} µg/m³{ENDC}")
    print(f"  {BOLD}RMSE:{ENDC}    {GREEN}{metrics['rmse']:.2f} µg/m³{ENDC}")
    print(f"  {BOLD}R²:{ENDC}      {GREEN}{metrics['r2']:.4f}{ENDC}")
    print(f"  {BOLD}MSE:{ENDC}     {metrics['mse']:.2f}")

    # Print prediction averages
    print(f"\n{BOLD}Prediction Summary:{ENDC}")
    print(f"  Average Predicted: {metrics['avg_predicted']:.2f} µg/m³")
    print(f"  Average Actual:    {metrics['avg_actual']:.2f} µg/m³")
    print(f"  Difference:        {abs(metrics['avg_predicted'] - metrics['avg_actual']):.2f} µg/m³")

    # Print model structure if available
    if hasattr(model, 'model') and model.model is not None:
        print(f"\n{BOLD}Model Structure:{ENDC}")
        model.model.summary(print_fn=lambda x: print(f"  {x}"))

    # Print footer
    print(f"\n{BOLD}{BLUE}═══════════════════════════════════════════════════════{ENDC}")
    print(f"{BOLD}{BLUE}                      END OF REPORT                    {ENDC}")
    print(f"{BOLD}{BLUE}═══════════════════════════════════════════════════════{ENDC}\n")


def rnn_ozone_model():
    """Main function to train and evaluate the RNN model."""
    # Create results directory
    results_dir = "results"
    model_dir = os.path.join(results_dir, "models")
    figures_dir = os.path.join(results_dir, "figures/rnn")
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(figures_dir, exist_ok=True)

    # Load processed data with all required features
    processed_dir = "data/processed"
    train_data = pd.read_csv(os.path.join(processed_dir, "train_data_full.csv"))
    test_data = pd.read_csv(os.path.join(processed_dir, "test_data_full.csv"))

    # Convert date column to datetime
    train_data['date'] = pd.to_datetime(train_data['date'])
    test_data['date'] = pd.to_datetime(test_data['date'])

    # Extract features and targets
    feature_cols = ['NO2', 'NOX', 'SO2', 'NMVOC', 'TEMP', 'RAD']
    # Add derived features like ones in the paper
    train_data['DayOfYear_Sin'] = np.sin(2 * np.pi * train_data['date'].dt.dayofyear / 365)
    train_data['DayOfYear_Cos'] = np.cos(2 * np.pi * train_data['date'].dt.dayofyear / 365)

    test_data['DayOfYear_Sin'] = np.sin(2 * np.pi * test_data['date'].dt.dayofyear / 365)
    test_data['DayOfYear_Cos'] = np.cos(2 * np.pi * test_data['date'].dt.dayofyear / 365)

    # Extended feature list with seasonality
    feature_cols_extended = feature_cols + ['DayOfYear_Sin', 'DayOfYear_Cos']

    X_train = train_data[feature_cols_extended]
    y_train = train_data['ozone'].values
    X_test = test_data[feature_cols_extended]
    y_test = test_data['ozone'].values

    # Print dataset information
    print("\nDataset Information:")
    print(f"Training samples: {len(X_train)}")
    print(f"Testing samples: {len(X_test)}")
    print(f"Features: {', '.join(feature_cols_extended)}")

    # Train LSTM model (as in Model 8/9 from the paper)
    print("\nTraining LSTM model...")
    lstm_model = OzoneRNNModel(rnn_type='lstm', n_features=len(feature_cols_extended),
                              sequence_length=1, name="Model8_LSTM")

    lstm_model.train(X_train, y_train, validation_split=0.2,
                    epochs=150, batch_size=32, patience=30)

    # Evaluate LSTM model
    print("\nEvaluating LSTM model...")
    lstm_metrics = lstm_model.evaluate(X_test, y_test)

    # Display pretty results
    display_pretty_results(lstm_model, lstm_metrics)

    # Create visualizations
    print("\nCreating visualization plots...")
    lstm_model.plot_results(X_test, y_test, test_data['date'],
                           save_dir=os.path.join(figures_dir, "lstm"))

    # Save model and metrics
    lstm_model.save_metrics(os.path.join(model_dir, "lstm_metrics.json"))
    lstm_model.save_model(model_dir)

    # Try GRU model as well (for comparison)
    print("\nTraining GRU model...")
    gru_model = OzoneRNNModel(rnn_type='gru', n_features=len(feature_cols_extended),
                             sequence_length=1, name="Model9_GRU")

    gru_model.train(X_train, y_train, validation_split=0.2,
                   epochs=150, batch_size=32, patience=30)

    # Evaluate GRU model
    print("\nEvaluating GRU model...")
    gru_metrics = gru_model.evaluate(X_test, y_test)

    # Display pretty results
    display_pretty_results(gru_model, gru_metrics)

    # Create visualizations
    print("\nCreating visualization plots...")
    gru_model.plot_results(X_test, y_test, test_data['date'],
                          save_dir=os.path.join(figures_dir, "gru"))

    # Save model and metrics
    gru_model.save_metrics(os.path.join(model_dir, "gru_metrics.json"))
    gru_model.save_model(model_dir)

    # Compare models
    print("\nModel Comparison:")
    comparison = {
        'LSTM': lstm_metrics,
        'GRU': gru_metrics
    }

    for model_name, metrics in comparison.items():
        print(f"{model_name}: MAE = {metrics['mae']:.2f} µg/m³, RMSE = {metrics['rmse']:.2f} µg/m³, R² = {metrics['r2']:.4f}")

    # Return the best model
    if lstm_metrics['mae'] <= gru_metrics['mae']:
        print(f"\nBest model: LSTM with MAE = {lstm_metrics['mae']:.2f} µg/m³")
        return lstm_model, lstm_metrics
    else:
        print(f"\nBest model: GRU with MAE = {gru_metrics['mae']:.2f} µg/m³")
        return gru_model, gru_metrics


if __name__ == "__main__":
    rnn_ozone_model()


Dataset Information:
Training samples: 2337
Testing samples: 585
Features: NO2, NOX, SO2, NMVOC, TEMP, RAD, DayOfYear_Sin, DayOfYear_Cos

Training LSTM model...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "Model8_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 1, 64)          │        18,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,649 (123.63 KB)

...

 Trainable params: 24,161 (94.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
59/59 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.5886 - mae: 0.6059 - val_loss: 0.1808 - val_mae: 0.3422 - learning_rate: 0.0010
Epoch 2/150
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1904 - mae: 0.3495 - val_loss: 0.1481 - val_mae: 0.3048 - learning_rate: 0.0010
Epoch 3/150
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1683 - mae: 0.3220 - val_loss: 0.1300 - val_mae: 0.2855 - learning_rate: 0.0010
Epoch 4/150
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1602 - mae: 0.3142 - val_loss: 0.1251 - val_mae: 0.2782 - learning_rate: 0.0010
Epoch 5/150
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1405 - mae: 0.2951 - val_loss: 0.1199 - val_mae: 0.2705 - learning_rate: 0.0010
Epoch 6/150
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1358 - mae: 0.2852 - val_loss: 0.1196 - val_mae: 0.2685 - learning_rate: 0.0010
Epoch 7/150
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1435 - mae: 0.2939 - val_loss: 0.1160 - val_mae: 0.2641 - learning_rate: 0.0010
Epoch 8/150


  Model: "Model9_GRU"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_1 (GRU)                     │ (None, 1, 64)          │        14,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 32)             │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            17 │
└─────────────────────────────────┴───────────────────

Model saved to results/models/Model9_GRU.h5

Model Comparison:
LSTM: MAE = 13.51 µg/m³, RMSE = 17.12 µg/m³, R² = 0.8361
GRU: MAE = 13.27 µg/m³, RMSE = 16.91 µg/m³, R² = 0.8401

Best model: GRU with MAE = 13.27 µg/m³


In [ ]:
"""
Convolutional Neural Network model for ozone concentration prediction.
Based on Model 11/12 from the research paper.
"""

# Suppress TensorFlow warnings
tf.get_logger().setLevel('ERROR')

class OzoneCNNModel:
    def __init__(self, n_features: int = 12, window_size: int = 7,
                 filters: int = 64, kernel_size: int = 3, name: str = "Model11_CNN"):
        """
        Initialize the CNN model for ozone prediction.

        Args:
            n_features (int): Number of input features
            window_size (int): Window size for convolution (days)
            filters (int): Number of filters in convolution layer
            kernel_size (int): Size of the convolution kernel
            name (str): Name of the model
        """
        self.n_features = n_features
        self.window_size = window_size
        self.filters = filters
        self.kernel_size = kernel_size
        self.name = name

        # Initialize model
        self.model = None

        # Initialize scalers
        self.feature_scaler = StandardScaler()
        self.target_scaler = StandardScaler()

        # Store metrics
        self.metrics = {}
        self.history = None

        # Feature names (will be set during training)
        self.feature_names = []

    def _build_model(self) -> None:
        """
        Build the CNN model architecture.
        """
        # Clear previous session to avoid conflicts
        tf.keras.backend.clear_session()

        # Define input shape based on window size and number of features
        input_shape = (self.window_size, self.n_features)

        # Create sequential model
        model = Sequential(name=self.name)

        # Add convolutional layers
        model.add(Conv1D(filters=self.filters, kernel_size=self.kernel_size,
                        activation='relu', padding='same',
                        input_shape=input_shape, name='conv_1'))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=2, name='pool_1'))

        model.add(Conv1D(filters=self.filters//2, kernel_size=self.kernel_size,
                        activation='relu', padding='same', name='conv_2'))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=2, name='pool_2'))

        # Flatten and add dense layers
        model.add(Flatten(name='flatten'))
        model.add(Dense(64, activation='relu', name='dense_1'))
        model.add(Dropout(0.2))
        model.add(Dense(32, activation='relu', name='dense_2'))
        model.add(Dense(1, activation='linear', name='output'))

        # Compile model
        model.compile(optimizer=Adam(learning_rate=0.001),
                      loss='mse',
                      metrics=['mae'])

        # Store model
        self.model = model

    def _prepare_windows(self, X: np.ndarray) -> np.ndarray:
        """
        Prepare input windows for CNN.

        Args:
            X (np.ndarray): Input features

        Returns:
            np.ndarray: Prepared windows
        """
        # Create sliding windows
        windows = []
        for i in range(len(X) - self.window_size + 1):
            windows.append(X[i:i + self.window_size])

        return np.array(windows)

    def train(self, X_train: pd.DataFrame, y_train: np.ndarray,
              validation_split: float = 0.2, epochs: int = 100,
              batch_size: int = 32, patience: int = 20) -> None:
        """
        Train the CNN model.

        Args:
            X_train (pd.DataFrame): Training features
            y_train (np.ndarray): Training targets (ozone concentration)
            validation_split (float): Fraction of training data to use for validation
            epochs (int): Maximum number of training epochs
            batch_size (int): Batch size for training
            patience (int): Patience for early stopping
        """
        # Store feature names
        self.feature_names = X_train.columns.tolist()
        self.n_features = len(self.feature_names)

        # Scale features
        X_scaled = self.feature_scaler.fit_transform(X_train)

        # Reshape y_train for scaler
        y_reshaped = y_train.reshape(-1, 1)
        y_scaled = self.target_scaler.fit_transform(y_reshaped).flatten()

        # Prepare input windows
        X_windows = self._prepare_windows(X_scaled)

        # For windowed data, we need to select corresponding targets
        y_windowed = y_scaled[self.window_size-1:]

        # Build the model
        self._build_model()

        # Print model summary
        self.model.summary()

        # Set up callbacks
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)
        ]

        # Train the model
        self.history = self.model.fit(
            X_windows, y_windowed,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=validation_split,
            callbacks=callbacks,
            verbose=1
        )

    def predict(self, X: pd.DataFrame) -> np.ndarray:
        """
        Make predictions using the trained model.

        Args:
            X (pd.DataFrame): Input features

        Returns:
            np.ndarray: Predicted ozone concentrations
        """
        # Check if model is built
        if self.model is None:
            raise ValueError("Model not trained yet. Call train() first.")

        # Check feature order
        missing_features = [feat for feat in self.feature_names if feat not in X.columns]
        if missing_features:
            raise ValueError(f"Missing features: {missing_features}")

        # Ensure correct feature order
        X = X[self.feature_names]

        # Scale features
        X_scaled = self.feature_scaler.transform(X)

        # Prepare windows
        X_windows = self._prepare_windows(X_scaled)

        # Make predictions
        y_pred_scaled = self.model.predict(X_windows)

        # Inverse transform to get original scale
        y_pred = self.target_scaler.inverse_transform(y_pred_scaled).flatten()

        # The predictions are for the last day of each window
        # We need to align with the original data, which means we're predicting
        # for indices starting at window_size-1
        # We'll pad the beginning with NaNs
        padded_predictions = np.full(len(X), np.nan)
        padded_predictions[self.window_size-1:] = y_pred

        return padded_predictions

    def evaluate(self, X_test: pd.DataFrame, y_test: np.ndarray) -> Dict[str, float]:
        """
        Evaluate the model on test data.

        Args:
            X_test (pd.DataFrame): Test features
            y_test (np.ndarray): True test targets

        Returns:
            Dict[str, float]: Dictionary containing evaluation metrics
        """
        # Make predictions
        y_pred = self.predict(X_test)

        # Get valid indices (non-NaN)
        valid_idx = ~np.isnan(y_pred)

        # Filter predictions and actual values
        y_pred_valid = y_pred[valid_idx]
        y_test_valid = y_test[valid_idx]

        # Calculate metrics
        mae = mean_absolute_error(y_test_valid, y_pred_valid)
        mse = mean_squared_error(y_test_valid, y_pred_valid)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test_valid, y_pred_valid)

        # Store metrics
        self.metrics = {
            'mae': float(mae),
            'mse': float(mse),
            'rmse': float(rmse),
            'r2': float(r2),
            'avg_predicted': float(np.mean(y_pred_valid)),
            'avg_actual': float(np.mean(y_test_valid)),
            'window_size': self.window_size,
            'filters': self.filters,
            'kernel_size': self.kernel_size,
            'n_features': self.n_features
        }

        return self.metrics

    def plot_results(self, X_test: pd.DataFrame, y_test: np.ndarray,
                    dates_test: pd.DatetimeIndex = None,
                    save_dir: str = "results/figures/cnn") -> None:
        """
        Create and save plots to visualize model performance.

        Args:
            X_test (pd.DataFrame): Test features
            y_test (np.ndarray): True test targets
            dates_test (pd.DatetimeIndex): Test dates for time series plot
            save_dir (str): Directory to save the plots
        """
        # Create save directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)

        # Make predictions
        y_pred = self.predict(X_test)

        # Get valid indices (non-NaN)
        valid_idx = ~np.isnan(y_pred)

        # Filter predictions and actual values
        y_pred_valid = y_pred[valid_idx]
        y_test_valid = y_test[valid_idx]

        # Filter dates if provided
        if dates_test is not None:
            dates_valid = dates_test[valid_idx]

        # 1. Training history plot
        if self.history is not None:
            plt.figure(figsize=(12, 5))

            # Plot training & validation loss
            plt.subplot(1, 2, 1)
            plt.plot(self.history.history['loss'], label='Training Loss')
            plt.plot(self.history.history['val_loss'], label='Validation Loss')
            plt.title('Model Loss During Training')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            plt.grid(alpha=0.3)

            # Plot training & validation MAE
            plt.subplot(1, 2, 2)
            plt.plot(self.history.history['mae'], label='Training MAE')
            plt.plot(self.history.history['val_mae'], label='Validation MAE')
            plt.title('Model MAE During Training')
            plt.xlabel('Epoch')
            plt.ylabel('MAE')
            plt.legend()
            plt.grid(alpha=0.3)

            plt.tight_layout()
            plt.savefig(os.path.join(save_dir, "training_history.png"))
            plt.close()

        # 2. Actual vs Predicted scatter plot
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test_valid, y_pred_valid, alpha=0.5)

        # Add perfect prediction line
        max_val = max(np.max(y_test_valid), np.max(y_pred_valid))
        min_val = min(np.min(y_test_valid), np.min(y_pred_valid))
        plt.plot([min_val, max_val], [min_val, max_val], 'r--')

        plt.xlabel('Actual Ozone Concentration (µg/m³)')
        plt.ylabel('Predicted Ozone Concentration (µg/m³)')
        plt.title('CNN Model: Actual vs Predicted Ozone')
        plt.grid(True, alpha=0.3)

        # Add metrics text
        r2 = self.metrics.get('r2', 0)
        rmse = self.metrics.get('rmse', 0)
        mae = self.metrics.get('mae', 0)

        metrics_text = f"R² = {r2:.3f}\nRMSE = {rmse:.2f} µg/m³\nMAE = {mae:.2f} µg/m³"
        plt.annotate(metrics_text, xy=(0.05, 0.95), xycoords='axes fraction',
                     verticalalignment='top',
                     bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        plt.savefig(os.path.join(save_dir, "actual_vs_predicted.png"))
        plt.close()

        # 3. Residual plot
        residuals = y_test_valid - y_pred_valid
        plt.figure(figsize=(10, 6))
        plt.scatter(y_pred_valid, residuals, alpha=0.5)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.xlabel('Predicted Ozone Concentration (µg/m³)')
        plt.ylabel('Residuals (µg/m³)')
        plt.title('Residual Plot')
        plt.grid(True, alpha=0.3)

        # Add residual statistics
        res_mean = np.mean(residuals)
        res_std = np.std(residuals)
        res_text = f"Mean = {res_mean:.2f} µg/m³\nStd Dev = {res_std:.2f} µg/m³"
        plt.annotate(res_text, xy=(0.05, 0.95), xycoords='axes fraction',
                     verticalalignment='top',
                     bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        plt.savefig(os.path.join(save_dir, "residual_plot.png"))
        plt.close()

        # 4. Time series plot (if dates are provided)
        if dates_test is not None:
            plt.figure(figsize=(15, 6))
            plt.plot(dates_valid, y_test_valid, label='Actual', alpha=0.7)
            plt.plot(dates_valid, y_pred_valid, label='Predicted', alpha=0.7)
            plt.xlabel('Date')
            plt.ylabel('Ozone Concentration (µg/m³)')
            plt.title('Ozone Concentration Over Time')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(os.path.join(save_dir, "time_series.png"))
            plt.close()

            # 5. Residuals over time
            plt.figure(figsize=(15, 6))
            plt.plot(dates_valid, residuals, color='royalblue', alpha=0.7)
            plt.axhline(y=0, color='r', linestyle='--')
            plt.xlabel('Date')
            plt.ylabel('Residuals (µg/m³)')
            plt.title('Prediction Residuals Over Time')
            plt.grid(True, alpha=0.3)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(os.path.join(save_dir, "residuals_time_series.png"))
            plt.close()

        # 6. Histogram of residuals
        plt.figure(figsize=(10, 6))
        sns.histplot(residuals, kde=True)
        plt.xlabel('Residuals (µg/m³)')
        plt.ylabel('Count')
        plt.title('Distribution of Residuals')
        plt.grid(True, alpha=0.3)
        plt.savefig(os.path.join(save_dir, "residual_distribution.png"))
        plt.close()

    def save_metrics(self, save_path: str) -> None:
        """
        Save model metrics to a JSON file.

        Args:
            save_path (str): Path to save the metrics
        """
        with open(save_path, 'w') as f:
            json.dump(self.metrics, f, indent=4)

    def save_model(self, save_dir: str) -> None:
        """
        Save the trained model.

        Args:
            save_dir (str): Directory to save the model
        """
        # Create directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)

        # Create model name
        model_path = os.path.join(save_dir, f"{self.name}.h5")

        # Save model
        self.model.save(model_path)

        # Save scalers
        import pickle
        with open(os.path.join(save_dir, f"{self.name}_feature_scaler.pkl"), 'wb') as f:
            pickle.dump(self.feature_scaler, f)

        with open(os.path.join(save_dir, f"{self.name}_target_scaler.pkl"), 'wb') as f:
            pickle.dump(self.target_scaler, f)

        print(f"Model saved to {model_path}")

    @classmethod
    def load_model(cls, model_dir: str, model_name: str) -> 'OzoneCNNModel':
        """
        Load a saved model.

        Args:
            model_dir (str): Directory where the model is saved
            model_name (str): Name of the model

        Returns:
            OzoneCNNModel: Loaded model
        """
        # Create model path
        model_path = os.path.join(model_dir, f"{model_name}.h5")

        # Load model
        loaded_model = tf.keras.models.load_model(model_path)

        # Load scalers
        import pickle
        with open(os.path.join(model_dir, f"{model_name}_feature_scaler.pkl"), 'rb') as f:
            feature_scaler = pickle.load(f)

        with open(os.path.join(model_dir, f"{model_name}_target_scaler.pkl"), 'rb') as f:
            target_scaler = pickle.load(f)

        # Create new instance
        instance = cls(name=model_name)
        instance.model = loaded_model
        instance.feature_scaler = feature_scaler
        instance.target_scaler = target_scaler

        # Try to extract window size and filters from model configuration
        for layer in loaded_model.layers:
            if isinstance(layer, tf.keras.layers.Conv1D):
                instance.filters = layer.filters
                instance.kernel_size = layer.kernel_size[0]
                instance.window_size = layer.input_shape[1]
                break

        return instance

def display_pretty_results(model, metrics, display_components=True):
    """
    Display model results in a nicely formatted way.

    Args:
        model: The trained model
        metrics: Dictionary containing model evaluation metrics
        display_components: Whether to display component coefficients
    """
    # Terminal formatting codes
    BOLD = '\033[1m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    ENDC = '\033[0m'

    # Print title
    print(f"\n{BOLD}{BLUE}═══════════════════════════════════════════════════════{ENDC}")
    print(f"{BOLD}{BLUE}           CNN OZONE PREDICTION MODEL                 {ENDC}")
    print(f"{BOLD}{BLUE}═══════════════════════════════════════════════════════{ENDC}\n")

    # Print model information
    print(f"{BOLD}Model Information:{ENDC}")
    print(f"  Type: Convolutional Neural Network")
    print(f"  Features: {metrics.get('n_features', 'Unknown')}")

    if 'window_size' in metrics:
        print(f"  Window Size: {metrics['window_size']}")
    if 'filters' in metrics:
        print(f"  Filters: {metrics['filters']}")
    if 'kernel_size' in metrics:
        print(f"  Kernel Size: {metrics['kernel_size']}")

    # Print key metrics
    print(f"\n{BOLD}Performance Metrics:{ENDC}")
    print(f"  {BOLD}MAE:{ENDC}     {GREEN}{metrics['mae']:.2f} µg/m³{ENDC}")
    print(f"  {BOLD}RMSE:{ENDC}    {GREEN}{metrics['rmse']:.2f} µg/m³{ENDC}")
    print(f"  {BOLD}R²:{ENDC}      {GREEN}{metrics['r2']:.4f}{ENDC}")
    print(f"  {BOLD}MSE:{ENDC}     {metrics['mse']:.2f}")

    # Print prediction averages
    print(f"\n{BOLD}Prediction Summary:{ENDC}")
    print(f"  Average Predicted: {metrics['avg_predicted']:.2f} µg/m³")
    print(f"  Average Actual:    {metrics['avg_actual']:.2f} µg/m³")
    print(f"  Difference:        {abs(metrics['avg_predicted'] - metrics['avg_actual']):.2f} µg/m³")

    # Print model structure if available
    if hasattr(model, 'model') and model.model is not None:
        print(f"\n{BOLD}Model Structure:{ENDC}")
        model.model.summary(print_fn=lambda x: print(f"  {x}"))

    # Print footer
    print(f"\n{BOLD}{BLUE}═══════════════════════════════════════════════════════{ENDC}")
    print(f"{BOLD}{BLUE}                      END OF REPORT                    {ENDC}")
    print(f"{BOLD}{BLUE}═══════════════════════════════════════════════════════{ENDC}\n")


def cnn_ozone_model():
    """Main function to train and evaluate the CNN model."""
    # Create results directory
    results_dir = "results"
    model_dir = os.path.join(results_dir, "models")
    figures_dir = os.path.join(results_dir, "figures/cnn")
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(figures_dir, exist_ok=True)

    # Load processed data with all required features
    processed_dir = "data/processed"
    train_data = pd.read_csv(os.path.join(processed_dir, "train_data_full.csv"))
    test_data = pd.read_csv(os.path.join(processed_dir, "test_data_full.csv"))

    # Convert date column to datetime
    train_data['date'] = pd.to_datetime(train_data['date'])
    test_data['date'] = pd.to_datetime(test_data['date'])

    # Extract features and targets
    feature_cols = ['NO2', 'NOX', 'SO2', 'NMVOC', 'TEMP', 'RAD']
    # Add derived features like ones in the paper
    train_data['DayOfYear_Sin'] = np.sin(2 * np.pi * train_data['date'].dt.dayofyear / 365)
    train_data['DayOfYear_Cos'] = np.cos(2 * np.pi * train_data['date'].dt.dayofyear / 365)

    test_data['DayOfYear_Sin'] = np.sin(2 * np.pi * test_data['date'].dt.dayofyear / 365)
    test_data['DayOfYear_Cos'] = np.cos(2 * np.pi * test_data['date'].dt.dayofyear / 365)

    # Extended feature list with seasonality
    feature_cols_extended = feature_cols + ['DayOfYear_Sin', 'DayOfYear_Cos']

    X_train = train_data[feature_cols_extended]
    y_train = train_data['ozone'].values
    X_test = test_data[feature_cols_extended]
    y_test = test_data['ozone'].values

    # Print dataset information
    print("\nDataset Information:")
    print(f"Training samples: {len(X_train)}")
    print(f"Testing samples: {len(X_test)}")
    print(f"Features: {', '.join(feature_cols_extended)}")

    # Train CNN model (for Model 11 from the paper)
    print("\nTraining CNN model...")
    cnn_model = OzoneCNNModel(n_features=len(feature_cols_extended),
                             window_size=7, filters=64, kernel_size=3,
                             name="Model11_CNN")

    cnn_model.train(X_train, y_train, validation_split=0.2,
                   epochs=150, batch_size=32, patience=30)

    # Evaluate CNN model
    print("\nEvaluating CNN model...")
    cnn_metrics = cnn_model.evaluate(X_test, y_test)

    # Display pretty results
    display_pretty_results(cnn_model, cnn_metrics)

    # Create visualizations
    print("\nCreating visualization plots...")
    cnn_model.plot_results(X_test, y_test, test_data['date'],
                          save_dir=os.path.join(figures_dir, "cnn"))

    # Save model and metrics
    cnn_model.save_metrics(os.path.join(model_dir, "cnn_metrics.json"))
    cnn_model.save_model(model_dir)

    # Try a different window size (for Model 12 from the paper)
    print("\nTraining CNN model with larger window...")
    cnn_model2 = OzoneCNNModel(n_features=len(feature_cols_extended),
                              window_size=14, filters=64, kernel_size=3,
                              name="Model12_CNN")

    cnn_model2.train(X_train, y_train, validation_split=0.2,
                    epochs=150, batch_size=32, patience=30)

    # Evaluate second CNN model
    print("\nEvaluating second CNN model...")
    cnn_metrics2 = cnn_model2.evaluate(X_test, y_test)

    # Display pretty results
    display_pretty_results(cnn_model2, cnn_metrics2)

    # Create visualizations
    print("\nCreating visualization plots...")
    cnn_model2.plot_results(X_test, y_test, test_data['date'],
                           save_dir=os.path.join(figures_dir, "cnn2"))

    # Save model and metrics
    cnn_model2.save_metrics(os.path.join(model_dir, "cnn2_metrics.json"))
    cnn_model2.save_model(model_dir)

    # Compare models
    print("\nModel Comparison:")
    print(f"CNN (Window={cnn_model.window_size}): MAE = {cnn_metrics['mae']:.2f} µg/m³, RMSE = {cnn_metrics['rmse']:.2f} µg/m³, R² = {cnn_metrics['r2']:.4f}")
    print(f"CNN (Window={cnn_model2.window_size}): MAE = {cnn_metrics2['mae']:.2f} µg/m³, RMSE = {cnn_metrics2['rmse']:.2f} µg/m³, R² = {cnn_metrics2['r2']:.4f}")

    # Return the best model
    if cnn_metrics['mae'] <= cnn_metrics2['mae']:
        print(f"\nBest model: CNN (Window={cnn_model.window_size}) with MAE = {cnn_metrics['mae']:.2f} µg/m³")
        return cnn_model, cnn_metrics
    else:
        print(f"\nBest model: CNN (Window={cnn_model2.window_size}) with MAE = {cnn_metrics2['mae']:.2f} µg/m³")
        return cnn_model2, cnn_metrics2


if __name__ == "__main__":
    cnn_ozone_model()


Dataset Information:
Training samples: 2337
Testing samples: 585
Features: NO2, NOX, SO2, NMVOC, TEMP, RAD, DayOfYear_Sin, DayOfYear_Cos

Training CNN model...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "Model11_CNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_1 (Conv1D)                 │ (None, 7, 64)          │         1,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 7, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_1 (MaxPooling1D)           │ (None, 3, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2 (Conv1D)                 │ (None, 3, 32)          │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 3, 32)          │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_2 (MaxPooling1D)           │ (None, 1, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,385 (48.38 KB)

...

 Trainable params: 16,289 (63.63 KB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/150
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.6107 - mae: 0.6273 - val_loss: 0.4910 - val_mae: 0.5376 - learning_rate: 0.0010
Epoch 2/150
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3432 - mae: 0.4672 - val_loss: 0.3989 - val_mae: 0.4872 - learning_rate: 0.0010
Epoch 3/150
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2788 - mae: 0.4156 - val_loss: 0.3913 - val_mae: 0.4829 - learning_rate: 0.0010
Epoch 4/150
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2428 - mae: 0.3896 - val_loss: 0.3682 - val_mae: 0.4629 - learning_rate: 0.0010
Epoch 5/150
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2338 - mae: 0.3851 - val_loss: 0.3439 - val_mae: 0.4521 - learning_rate: 0.0010
Epoch 6/150
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2011 - mae: 0.3530 - val_loss: 0.2595 - val_mae: 0.4016 - learning_rate: 0.0010
Epoch 7/150
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1934 - mae: 0.3500 - val_loss: 0.2699 - val_mae: 0.4118 - learning_rate: 0.0010
Epoch 8/150
5

  Model: "Model12_CNN"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_1 (Conv1D)                 │ (None, 14, 64)         │         1,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 14, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_1 (MaxPooling1D)           │ (None, 7, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2 (Conv1D)                 │ (None, 7, 32)          │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 7, 32)    

Model saved to results/models/Model12_CNN.h5

Model Comparison:
CNN (Window=7): MAE = 14.46 µg/m³, RMSE = 18.21 µg/m³, R² = 0.8155
CNN (Window=14): MAE = 14.22 µg/m³, RMSE = 17.98 µg/m³, R² = 0.8213

Best model: CNN (Window=14) with MAE = 14.22 µg/m³
